# Convert source ids from one region to another

In [1]:
from astropy import units as uu
#First region, usually the old one
region1_name = "ALL_10-800"  #"ALL_10-800" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region1_fname = "./vasca_pipeline/"+region1_name+"/region_"+region1_name+".fits"

#Source IDs to be converted
srcs_ids1 =  [2996 ,11821 ,55034 ,61491 ,64360 ,73354 ,76243 ,92867 ,104078 ,106745 ,124860 ,162490 ,182622 ,199884 ,226975 ,267514 ,588567 ,599887 ,789511 ,814134 ,] # WD and candidates, including GAIA info

#Second region, usually the new one
region2_name = "ALL_10-800_LOOSE"
region2_fname = "./vasca_pipeline/"+region2_name+"/region_"+region2_name+".fits"

#Maximum distance to be considered the same source
dist_max = 1*uu.arcsec

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
#logger.enable("vasca")

In [3]:
rg1 = Region()
rg1.load_from_fits(region1_fname)
rg2 = Region()
rg2.load_from_fits(region2_fname)

if len(srcs_ids1)>0:
    rg1.tt_sources.add_index("rg_src_id")
    idx1_srcs = rg1.tt_sources.loc_indices["rg_src_id", srcs_ids1]
    tt_srcs1 = Table(rg1.tt_sources[idx1_srcs])
else:
    tt_srcs1 = rg.tt_sources
    
coords1 = SkyCoord(tt_srcs1["ra"], tt_srcs1["dec"], frame="icrs")
coords2 = SkyCoord(rg2.tt_sources["ra"], rg2.tt_sources["dec"], frame="icrs")
idx1, d2d, d3d = coords2.match_to_catalog_sky(coords1)

rg_src_id1 = tt_srcs1[idx1]["rg_src_id"]
rg2.tt_sources["rg_src_id1"]=rg_src_id1
rg2.tt_sources["dist_id1"]=d2d.to(uu.arcsec)
sel = rg2.tt_sources["dist_id1"] < dist_max
#display(rg2.tt_sources["rg_src_id","rg_src_id1","dist_id1"][sel])
rg2.tt_sources["rg_src_id","rg_src_id1","dist_id1"][sel].pprint_all()
print("srcs_ids = [", end="")
for src_id in rg2.tt_sources["rg_src_id"][sel].data:
    print(src_id,",", end="")
print("]")
print(sel.sum(), "counterparts found, for", len(srcs_ids1), "sources" )

rg_src_id rg_src_id1        dist_id1       
                             arcsec        
--------- ---------- ----------------------
     8215       2996  0.0006461470132057625
    32448      11821  5.868269211343778e-11
    67711      92867    0.21070969843371945
   151796      55034    0.03358798928417413
   166179      61491                    0.0
   172775      64360     0.0200684618342995
   193067      73354 1.4339030727067295e-10
   199832      76243   0.017188398987915875
   259271     104078   0.023469826296120298
   265150     106745   0.024960750174426984
   305192     124860   0.013248172871326059
   388172     162490     0.1233475292268099
   432606     182622    0.07792410812055205
   472623     199884  5.126678565123436e-11
   535864     226975    0.19300960416390975
   625693     267514  6.057443324924953e-10
  1521738     588567    0.11106603209001159
  1551422     599887                    0.0
  2136829     789511     0.2591118790872122
  2210890     814134     0.85956